In [ ]:
from main import main
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
from matplotlib import rcParams
import time

rcParams['animation.embed_limit'] = 1000  # Größere Animationen einbetten

size = main()  # Ruft die `main()` Funktion auf und speichert die Größe der Welt

# Lade die Simulationsdaten
with open("simulation_data.json", "r") as file:
    data = json.load(file)

# Extrahiere die Zeitschritte und Anzahl der Körper
timesteps = sorted(map(int, data.keys()))
num_bodies = len(data[str(timesteps[0])]["positions"][0])  # Anzahl der Körper

# Konvertiere JSON-Daten in NumPy-Arrays
coordinates = [
    np.array(data[str(t)]["positions"]).T for t in timesteps
]

# Initialisiere das 3D-Plot-Fenster
fig = plt.figure(figsize=(8, 8))
ax_scatter = fig.add_subplot(111, projection="3d")

# Scatter-Plot für die aktuellen Positionen der Körper
scatter = ax_scatter.scatter([], [], [], s=50, c="blue", animated=True)

# Liste, um die Spuren (Trails) der Körper zu speichern
trails = [[] for _ in range(num_bodies)]  # Eine Liste für jeden Körper

# Liste, um die Linienobjekte der Spuren zu speichern
trail_lines = [None for _ in range(num_bodies)]

# Setze die Achsenlimits basierend auf `size`
ax_scatter.set_xlim(-size, size)
ax_scatter.set_ylim(-size, size)
ax_scatter.set_zlim(-size, size)

ax_scatter.set_xlabel("X-coordinate")
ax_scatter.set_ylabel("Y-coordinate")
ax_scatter.set_zlabel("Z-coordinate")
ax_scatter.set_title("3D-Simulation Animation")

# Update-Funktion für die Animation
def update(frame):
    """Aktualisiert die Positionen der Körper für jeden Zeitschritt."""
    points = coordinates[frame]

    # Aktualisiere die Positionen der Körper
    scatter._offsets3d = (points[:, 0], points[:, 1], points[:, 2])  # X, Y, Z aktualisieren

    # Lösche die alten Spuren
    for i in range(num_bodies):
        if trail_lines[i] is not None:
            trail_lines[i].remove()
            trail_lines[i] = None

    # Füge die aktuellen Positionen zu den Spuren hinzu
    for i in range(num_bodies):
        trails[i].append(points[i])  # Füge die aktuelle Position zur Spur hinzu

        if len(trails[i]) > 200:
            trails[i] = trails[i][-200:]  # Behalte nur die letzten 200 Punkte

        # Zeichne die Spur für jeden Körper
        if len(trails[i]) > 1:  # Nur zeichnen, wenn es mindestens zwei Punkte gibt
            x = [p[0] for p in trails[i]]
            y = [p[1] for p in trails[i]]
            z = [p[2] for p in trails[i]]
            trail_lines[i] = ax_scatter.plot(x, y, z, color="gray", alpha=0.5, linewidth=1)[0]

    ax_scatter.set_title(f"Time Step {frame}")
    return scatter,

print("Creating animation...")
start_time_animation = time.time()

# Erstelle die Animation
ani = FuncAnimation(fig, update, frames=len(timesteps), blit=False, interval=24)

plt.close()  # Verhindert doppelte Anzeige

# Animation in Jupyter anzeigen
display(HTML(ani.to_jshtml()))

end_time_animation = time.time()
print(f"I took {round(end_time_animation - start_time_animation, 2)} seconds to create the animation.")

In [ ]:
from matplotlib.animation import FFMpegWriter

mp4_filename = "simulation_animation.mp4"
writer = FFMpegWriter(fps=60, metadata={'title': 'Simulation Animation', 'artist': 'Matplotlib'})

print("Saving animation as MP4...")
ani.save(mp4_filename, writer=writer)  # saves the animation into .mp4
print(f"MP4 saved as {mp4_filename}")